# Explore new dataset from Gießen

Preprocessing. 

get some descriptive stats on the sample we have here. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pdb

pd.set_option('display.max_columns', None)

cur_dir = os.getcwd()
path_to_data = os.path.join(cur_dir, "data_trustgame_depression")
os.chdir(path_to_data)

In [2]:
# negative to positive data
df_np = pd.read_csv('trustgame_neg_to_pos.csv')
df_pn = pd.read_csv('trustgame_pos_to_neg.csv')
df_a  = pd.read_csv('trustgame_ambiguous.csv')

In [3]:
# start cleaning this data. Start with df_np bc we use this for DPK a bit. 
# drop unneeded cols
unwanted_cols = ['Event Index', 'Participant Status','Task Version', 'UTC Timestamp', 'UTC Date and Time', 'Local Timestamp',
       'Local Timezone', 'Local Date and Time', 'Experiment ID','Participant Public ID',
       'Experiment Version', 'Tree Node Key','Spreadsheet Name','Task Name', 'Repeat Key', 'Image','Schedule ID','practice trial keep', 'practice trial share',
       'Participant External Session ID', 'Participant Completion Code', 'Reaction Onset','Response Type',
       'Participant Device Type', 'Participant Device', 'Participant OS','Dishonest',
       'Participant Browser', 'Participant Monitor Size', 'Participant Starting Group',
       'Participant Viewport Size', 'Checkpoint', 'Room ID', 'Room Order','Zone Name',
       'Attempt','Correct','Incorrect','Correct answer','KeepAmount','Timelag_decision','Timelag_next participant',
       'checkpoint-jpdv', 'randomiser-i3f2', 'X Coordinate', 'Y Coordinate','randomise_blocks','randomise_trials',
       'checkpoint-neje', 'order-73e3', 'checkpoint-iq13', 'checkpoint-t4s8','Timed Out', 'ANSWER']

df_np = df_np.drop(unwanted_cols, axis=1)
df_pn = df_pn.drop(unwanted_cols, axis=1)
df_a  = df_a.drop(unwanted_cols, axis=1)

In [4]:
# now we start filtering. 
df_np = df_np[df_np['display']=='Trials']
df_pn = df_pn[df_pn['display']=='Trials']
df_a  = df_a[df_a['display']=='Trials']
print(df_np.shape, df_pn.shape, df_a.shape)
df_np = df_np[df_np['Zone Type']=='response_button_text']
df_pn = df_pn[df_pn['Zone Type'] == 'response_button_text']
df_a  = df_a[df_a['Zone Type'] == 'response_button_text']
print(df_np.shape, df_pn.shape, df_a.shape)

(1542, 14) (2229, 14) (1797, 14)
(420, 14) (600, 14) (480, 14)


In [5]:
vp_pn = list(df_pn['Participant Private ID'].unique())
vp_np = list(df_np['Participant Private ID'].unique())
vp_a  = list(df_a['Participant Private ID'].unique())
print(f'N pos-neg: {len(vp_pn)}')
print(f'N neg-pos: {len(vp_np)}')
print(f'ambiguous: {len(vp_a)}')

N pos-neg: 10
N neg-pos: 7
ambiguous: 8


In [6]:
# merge all of them
df_list = [df_np, df_pn, df_a]
df = pd.concat(df_list)

In [7]:
# now we replace values so we can work with it. 
df['Response'] = df['Response'].replace({'50ct spenden': 1, '50ct behalten': 0}) # now 1 = spenden, 0 = behalten
df['partnerAnswer'] = df['partnerAnswer'].replace({'Entscheidung von Mitspieler(in): "Nichts zurückgeben"': 1, 
                                                  'Entscheidung von Mitspieler(in): "1€ zurückgeben"': 0}) # 0: abuse, 1: coop

# reward and behavior of person need to be the same! abuse = 1, social = 0
# partnerAnswer muss 0 social, 1 antiscoial, 2 unknown
# reward 0 return, 1 loss, 2 keep same amount
# response 0 für share, 1 für keep, 2 für start ganz am Anfang der Daten. 

# list of all subjects
all_vp = list(df['Participant Private ID'].unique())
print(f'All VP: {len(all_vp)}')

All VP: 25


In [8]:
# create a reward column.
reward_conditions = [
    (df['Response'] == 0) & (df['partnerAnswer']==0), 
    (df['Response'] == 0) & (df['partnerAnswer']==1), 
    (df['Response'] == 1)
]

# response 0, partner 0 = outcome 0 (return). 
# response 0, partner 1 = outcome 1 (loss). 
# response 1, partner irrelevant = outcome 2 (keep)

choices = [0, 1, 2]
df['reward'] = np.select(reward_conditions, choices, default=0)

# constraint: if response == 1, reward & partnerAnswer need to be 2.
# response: [share, keep, start]. partnerAnswer: [social, antisocial, unknown]. reward: [return, loss, keep]
df.loc[df['Response'] == 1, ['reward', 'partnerAnswer']] = 2

In [9]:
def test_dataframe_conditions(df):
    # Test 1: response = 0 [share] & partnerAnswer = 0 [return], reward needs to be 0 (WIN)
    assert all(df[(df['Response'] == 0) & (df['partnerAnswer'] == 0)]['reward'] == 0), "Test 1 failed"
    
    # Test 2: response = 0 [share] & partnerAnswer = 1 [defect], reward needs to be 1 (LOSS)
    assert all(df[(df['Response'] == 0) & (df['partnerAnswer'] == 1)]['reward'] == 1), "Test 2 failed"
    
    # Test 3: response = 1 [keep], reward needs to be 2 [budget keep]
    assert all(df[df['Response'] == 1]['reward'] == 2), "Test 3 failed"
    
test_dataframe_conditions(df)
#df.to_csv('final_data.csv')

# BDI data preprocessing

In [10]:
# preprocess the BDI survey data.
df_bdi = pd.read_csv('data_bdi.csv')

survey_unwanted_cols = ['Event Index', 'UTC Timestamp', 'UTC Date and Time', 'Local Timestamp',
       'Local Timezone', 'Local Date and Time', 'Experiment ID',
       'Experiment Version', 'Tree Node Key', 'Repeat Key', 'Schedule ID',
       'Participant Public ID', 'Participant Starting Group', 'Participant Status',
       'Participant Completion Code', 'Participant External Session ID',
       'Participant Device Type', 'Participant Device', 'Participant OS',
       'Participant Browser', 'Participant Monitor Size',
       'Participant Viewport Size', 'Checkpoint', 'Room ID', 'Room Order','checkpoint-jpdv', 'randomiser-i3f2',
       'checkpoint-neje', 'order-73e3', 'checkpoint-iq13', 'checkpoint-t4s8',
       'Randomise questionnaire elements?']

df_bdi = df_bdi.drop(survey_unwanted_cols, axis=1)

In [11]:
# items
bdi_items = ['BDI_' + f'0{i}' for i in range(1,10)] + ['BDI_' + f'{i}' for i in range(10, 22)]
# responses
bdi_resp = [i + '-quantised' for i in bdi_items]
# all participants
vp_bdi = list(df_bdi['Participant Private ID'].unique())

# preprocess survey data to make it easier to use
# Drop rows with NaN values in the "Question Key" column
df_bdi = df_bdi.dropna(subset=["Question Key"])

# Filter out rows that are not relevant
df_bdi = df_bdi[df_bdi["Question Key"].str.startswith("BDI")]
df_items = df_bdi[~df_bdi["Question Key"].str.contains("quantised")]
df_quantised = df_bdi[df_bdi["Question Key"].str.contains("quantised")]

In [12]:
df_items["Question Number"] = df_items["Question Key"].str.replace("BDI_", "").astype(int)
df_quantised["Question Number"] = df_quantised["Question Key"].str.replace("BDI_", "").str.replace("-quantised", "").astype(int)

C:\Users\annae\AppData\Local\Temp\ipykernel_7872\4061585974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items["Question Number"] = df_items["Question Key"].str.replace("BDI_", "").astype(int)
C:\Users\annae\AppData\Local\Temp\ipykernel_7872\4061585974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quantised["Question Number"] = df_quantised["Question Key"].str.replace("BDI_", "").str.replace("-quantised", "").astype(int)


In [13]:
# restructure the whole dataframe
df_merged = pd.merge(df_items, df_quantised, on=["Participant Private ID", "Task Name", "Task Version", "Question Number"], suffixes=('_item', '_quantised'))
final_df = df_merged[["Participant Private ID", "Task Name", "Task Version", "Question Key_item", "Response_item", "Response_quantised"]]
final_df.columns = ["Participant Private ID", "Task Name", "Task Version", "Question Key", "Item", "Response"]
df_bdi = final_df

print('BDI data for N = ',len(vp_bdi))
vp_bdi.pop(-1) # remove nan value

BDI data for N =  26


nan

In [14]:
bdi_res = dict()
for vp in vp_bdi: 
    df_sbj = df_bdi[df_bdi['Participant Private ID'] == vp]
    vp_score = df_sbj['Response'].astype(int).sum() -21 # minus 21 bc I think it is scored 0-4
    bdi_res[vp] = vp_score
    
bdi_median = np.median(list(bdi_res.values()))
print('median value BDI: ', bdi_median)

bdi = pd.DataFrame(list(bdi_res.items()), columns=['sbj_id', 'bdi_score'])
bdi.to_csv('bdi_n17.csv')

# add BDI value to the dataframe. 
df['BDI'] = df['Participant Private ID'].map(bdi_res)
# run test to be sure... 
test_dataframe_conditions(df)
# if not failed, put into csv file. 
df.to_csv('data_inclBDI_n25.csv')

median value BDI:  22.0


In [19]:
# special purpose datafiles: only ambiguous..
df_a = df[df['Spreadsheet Name']=='Ambigious']
df_a.to_csv('data_ambiguous_only.csv')

In [15]:
df.head()

,Participant Private ID,Spreadsheet,Spreadsheet Row,Trial Number,Screen Number,Screen Name,Zone Type,Reaction Time,Response,display,Button A,Button B,Zuweisung_Mitspieler(in),partnerAnswer,reward,BDI
14,9089012.0,Negative-to-positive,17.0,1,1.0,decision,response_button_text,4560.0,1,Trials,50ct behalten,50ct spenden,"Sie spielen mit ""keEpcalm""",2,2,10
22,9089012.0,Negative-to-positive,7.0,2,1.0,decision,response_button_text,2183.0,1,Trials,50ct behalten,50ct spenden,"Sie spielen mit ""sdlfn""",2,2,10
30,9089012.0,Negative-to-positive,12.0,3,1.0,decision,response_button_text,3546.0,1,Trials,50ct behalten,50ct spenden,"Sie spielen mit ""schnuck""",2,2,10
38,9089012.0,Negative-to-positive,25.0,4,1.0,decision,response_button_text,508.0,0,Trials,50ct behalten,50ct spenden,"Sie spielen mit ""ines""",1,1,10
44,9089012.0,Negative-to-positive,23.0,5,1.0,decision,response_button_text,507.0,0,Trials,50ct behalten,50ct spenden,"Sie spielen mit ""8D""",0,0,10
